Project 3 Part 1 (Core)

Business Problem For this project, you have been hired to produce a MySQL database on Movies from a subset of IMDB's publicly available dataset. Ultimately, you will use this database to analyze what makes a movie successful and will provide recommendations to the stakeholder on how to make a successful movie.

In [1]:
#import the package
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
!pip install tmdbsimple
import os, json, math, time
from tqdm.notebook import tqdm_notebook

In [2]:
# upload the files
basics = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', 
                     sep='\t', low_memory=False)

In [3]:
ratings = pd.read_csv('https://datasets.imdbws.com/title.ratings.tsv.gz', 
                      sep='\t', low_memory=False)

In [6]:
akas = pd.read_csv('https://datasets.imdbws.com/title.akas.tsv.gz', 
                   sep='\t', low_memory=False)

In [7]:
# replace the bad spelling missing values with Nan
basics = basics.replace({'\\N':np.nan}) 
ratings = ratings.replace({'\\N':np.nan}) 
akas = akas.replace({'\\N':np.nan}) 

In [10]:
# getting just the movies
basics = basics.loc[basics['titleType'] == 'movie']

In [12]:
# deliting the missing values
basics = basics.dropna(subset = ['runtimeMinutes', 'genres','startYear'])

In [14]:
# loc the dates the request from us 
basics['startYear'] = basics['startYear'].astype(int)
basics = basics.loc[(basics['startYear'] >= 2000) 
                    & (basics['startYear'] <= 2022)]

C:\Users\raged\anaconda3\envs\dojo-env\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
# taking out the documentary from our data
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]

In [20]:
basics['genres'].unique()

array(['Comedy,Fantasy,Romance', 'Drama', 'Drama,War',
       'Comedy,Horror,Sci-Fi', 'Comedy', 'Comedy,Drama,Fantasy',
       'Drama,Romance', 'Comedy,Mystery', 'Drama,Fantasy', 'Adventure',
       'Musical,Romance', 'Action,Adventure,Drama', 'Action',
       'Crime,Thriller', 'Comedy,Fantasy', 'Action,Crime,Drama',
       'Action,Thriller', 'Comedy,Drama,Romance', 'Drama,Music,Romance',
       'Comedy,Horror,Mystery', 'Crime,Drama,Thriller', 'Comedy,Drama',
       'Action,Adventure,Animation', 'Comedy,Romance', 'Drama,Thriller',
       'Comedy,Drama,Sci-Fi', 'Adventure,Family,Fantasy', 'Drama,History',
       'Drama,History,War', 'Adventure,Animation,Comedy',
       'Action,Adventure,Fantasy', 'Action,Drama,Sci-Fi',
       'Biography,Drama,Romance', 'Horror,Mystery,Thriller',
       'Comedy,Drama,Thriller', 'Animation,Family,Musical',
       'Drama,Mystery,Thriller', 'Action,Adventure,Thriller',
       'Action,Horror,Sci-Fi', 'Action,Adventure,Sci-Fi',
       'Action,Adventure,Comedy

In [21]:
# just us movies
akas = akas.loc[akas['region'] == 'US']

In [22]:
keepers = basics['tconst'].isin(akas['titleId'])

In [24]:
basics= basics[keepers]

In [25]:
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34790,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61089,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
67634,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
86765,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
92730,tt0094859,movie,Chief Zabu,Chief Zabu,0,2016,NaN,74,Comedy
...,...,...,...,...,...,...,...,...,...
9151011,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019,NaN,74,Drama
9151407,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019,NaN,97,"Comedy,Drama,Fantasy"
9151547,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,NaN,51,Drama
9151556,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller"
